In [1]:
import requests
from dotenv import load_dotenv
import os
import base64
import json
import csv

In [2]:
load_dotenv()
# replace with your eBay application credentials
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

# obtain OAuth 2.0 access token
def get_access_token(client_id, client_secret):
    url = 'https://api.ebay.com/identity/v1/oauth2/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode(),
    }
    data = {
        'grant_type': 'client_credentials',
        'scope': 'https://api.ebay.com/oauth/api_scope'
    }
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    return response.json()['access_token']

# search for computers on eBay
def search_computers(access_token, country_code):
    url = 'https://api.ebay.com/buy/browse/v1/item_summary/search'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'q': 'computer',
        'filter': f'deliveryCountry:{country_code}'
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()


In [3]:
# Define the mapping of eBay marketplaces
countries = {
    'eBay United States': ('EBAY_US', 'US'),
    'eBay Canada (English)': ('EBAY_CA', 'CA'),
    'eBay UK': ('EBAY_GB', 'GB'),
    'eBay Australia': ('EBAY_AU', 'AU'),
    'eBay Austria': ('EBAY_AT', 'AT'),
    'eBay Belgium (French)': ('EBAY_FRBE', 'BE'),
    'eBay France': ('EBAY_FR', 'FR'),
    'eBay Germany': ('EBAY_DE', 'DE'),
    'eBay Italy': ('EBAY_IT', 'IT'),
    'eBay Belgium (Dutch)': ('EBAY_NLBE', 'BE'),
    'eBay Netherlands': ('EBAY_NL', 'NL'),
    'eBay Spain': ('EBAY_ES', 'ES'),
    'eBay Switzerland': ('EBAY_CH', 'CH'),
    'eBay Hong Kong': ('EBAY_HK', 'HK'),
    'eBay Ireland': ('EBAY_IE', 'IE'),
    'eBay Canada (French)': ('EBAY_FRCA', 'CA'),
    'eBay Poland': ('EBAY_PL', 'PL'),
    'eBay Singapore': ('EBAY_SG', 'SG')
}

18


In [6]:
csv_file = "data.csv"
first_write = True  # Flag pour savoir si on doit écrire l'en-tête

for site_name, (marketplace_id, country_code) in countries.items():
    try:
        # retrieve access token for the specific marketplace
        access_token = get_access_token(client_id, client_secret)
        json_data = search_computers(access_token, country_code)

        # extract relevant fields
        filtered_data = []
        for item in json_data.get("itemSummaries", []):
            filtered_data.append({
                "Item ID": item.get("itemId"),
                "Title": item.get("title"),
                "Price": item.get("price", {}).get("value"),
                "Currency": item.get("price", {}).get("currency"),
                "Condition": item.get("condition"),
                "Item URL": item.get("itemWebUrl"),
                "Image URL": item.get("image", {}).get("imageUrl"),
                "Seller Username": item.get("seller", {}).get("username"),
                "Seller Feedback %": item.get("seller", {}).get("feedbackPercentage"),
                "Seller Feedback Score": item.get("seller", {}).get("feedbackScore"),
                "Shipping Cost": item.get("shippingOptions", [{}])[0].get("shippingCost", {}).get("value"),
                "Shipping Currency": item.get("shippingOptions", [{}])[0].get("shippingCost", {}).get("currency"),
                "Original Price": item.get("marketingPrice", {}).get("originalPrice", {}).get("value"),
                "Discount Percentage": item.get("marketingPrice", {}).get("discountPercentage"),
                "Discount Amount": item.get("marketingPrice", {}).get("discountAmount", {}).get("value"),
                "Categories": ", ".join([cat.get("categoryName", "Unknown") if isinstance(cat, dict) else str(cat) for cat in item.get("categories", [])]),
                "Country": site_name
            })

        # write data to CSV
        if filtered_data:
            mode = "w" if first_write else "a"  # "w" for the first write, "a" for append
            with open(csv_file, mode, newline="", encoding="utf-8") as file:
                writer = csv.DictWriter(file, fieldnames=filtered_data[0].keys())
                if first_write:
                    writer.writeheader()  # write header only once
                    first_write = False  # disable after the first write
                writer.writerows(filtered_data)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred for marketplace {marketplace_id}: {e}")

In [163]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head()

,Item ID,Title,Price,Currency,Condition,Item URL,Image URL,Seller Username,Seller Feedback %,Seller Feedback Score,Shipping Cost,Shipping Currency,Original Price,Discount Percentage,Discount Amount,Categories,Leaf Category IDs,Country
0,v1|304762827171|605112036398,"HP Desktop Computer PC i7, up to 64GB RAM, 4TB...",348.06,USD,Excellent - Refurbished,https://www.ebay.com/itm/304762827171?_skw=com...,https://i.ebayimg.com/images/g/NG4AAOSwyt1nqav...,tech_leaf,100.0,14745,0.0,USD,1740.29,80.0,1392.23,"PC Desktops & All-In-Ones, Computers/Tablets &...",179,eBay United States
1,v1|304041042675|605111979988,"Dell Desktop Computer PC i5, up to 32GB RAM, 3...",289.00,USD,Excellent - Refurbished,https://www.ebay.com/itm/304041042675?_skw=com...,https://i.ebayimg.com/images/g/8MIAAOSwQKpnrFd...,tech_leaf,100.0,14745,0.0,USD,1445.00,80.0,1156.00,"PC Desktops & All-In-Ones, Computers/Tablets &...",179,eBay United States
2,v1|365213672465|634885076951,"DELL Desktop Computer PC i7, up to 64GB RAM, 1...",161.70,USD,Excellent - Refurbished,https://www.ebay.com/itm/365213672465?_skw=com...,https://i.ebayimg.com/images/g/GBEAAOSwuShnRNz...,kenji2007,99.2,2206,0.0,USD,770.00,79.0,608.30,"PC Desktops & All-In-Ones, Computers/Tablets &...",179,eBay United States
3,v1|365297986474|635060753594,"DELL Desktop Computer PC i7, up to 64GB RAM 16...",355.95,USD,Excellent - Refurbished,https://www.ebay.com/itm/365297986474?_skw=com...,https://i.ebayimg.com/images/g/~ewAAOSwzLlnZd~...,kenji2007,99.2,2206,0.0,USD,1695.00,79.0,1339.05,"PC Desktops & All-In-Ones, Computers/Tablets &...",179,eBay United States
4,v1|315248241753|0,HP EliteDesk 800 G3 35W Mini PC Computer - Int...,58.49,USD,Used,https://www.ebay.com/itm/315248241753?_skw=com...,https://i.ebayimg.com/images/g/Ur4AAOSwb1Fnr0f...,mr.itparts,99.9,36320,0.0,USD,NaN,NaN,NaN,"PC Desktops & All-In-Ones, Computers/Tablets &...",179,eBay United States


In [164]:
data.dtypes

Item ID                   object
Title                     object
Price                    float64
Currency                  object
Condition                 object
Item URL                  object
Image URL                 object
Seller Username           object
Seller Feedback %        float64
Seller Feedback Score      int64
Shipping Cost            float64
Shipping Currency         object
Original Price           float64
Discount Percentage      float64
Discount Amount          float64
Categories                object
Leaf Category IDs         object
Country                   object
dtype: object

In [165]:
# we drop unuseful collumns
col_to_drop = ['Item ID', 'Item URL', 'Image URL', 'Seller Username', 'Leaf Category IDs']
data.drop(columns=col_to_drop, inplace=True)
data.head()

,Title,Price,Currency,Condition,Seller Feedback %,Seller Feedback Score,Shipping Cost,Shipping Currency,Original Price,Discount Percentage,Discount Amount,Categories,Country
0,"HP Desktop Computer PC i7, up to 64GB RAM, 4TB...",348.06,USD,Excellent - Refurbished,100.0,14745,0.0,USD,1740.29,80.0,1392.23,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States
1,"Dell Desktop Computer PC i5, up to 32GB RAM, 3...",289.00,USD,Excellent - Refurbished,100.0,14745,0.0,USD,1445.00,80.0,1156.00,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States
2,"DELL Desktop Computer PC i7, up to 64GB RAM, 1...",161.70,USD,Excellent - Refurbished,99.2,2206,0.0,USD,770.00,79.0,608.30,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States
3,"DELL Desktop Computer PC i7, up to 64GB RAM 16...",355.95,USD,Excellent - Refurbished,99.2,2206,0.0,USD,1695.00,79.0,1339.05,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States
4,HP EliteDesk 800 G3 35W Mini PC Computer - Int...,58.49,USD,Used,99.9,36320,0.0,USD,NaN,NaN,NaN,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States


In [166]:
# display missings
data.isnull().sum()

Title                      0
Price                      0
Currency                   0
Condition                  9
Seller Feedback %          0
Seller Feedback Score      0
Shipping Cost             16
Shipping Currency         16
Original Price           712
Discount Percentage      666
Discount Amount          712
Categories                 0
Country                    0
dtype: int64

In [167]:
# for original missings are informaion that mean the user doesnt provide the original price or he/she doesnt discount the price, lets replase it with current price
# for currency missings are 'USD'
# for shipping cost we use knn from sklearn
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

data['Original Price'].fillna(data['Price'], inplace=True)
data['Discount Amount'].fillna(0, inplace=True)
data['Shipping Currency'].fillna(data['Currency'], inplace=True)

# fill 'Discount Percentage'
data["Discount Percentage"] = data.apply(
    lambda row: round(((row["Original Price"] - row["Price"]) * 100 / row["Original Price"])) if pd.notna(row["Original Price"]) and row["Original Price"] > 0 else 0,
    axis=1,
)

data['Shipping Cost'] = pd.to_numeric(data['Shipping Cost'], errors='coerce')
imputer = KNNImputer(n_neighbors=3)
data[['Shipping Cost']] = imputer.fit_transform(data[['Shipping Cost']])

print(data.isnull().sum())


Title                    0
Price                    0
Currency                 0
Condition                9
Seller Feedback %        0
Seller Feedback Score    0
Shipping Cost            0
Shipping Currency        0
Original Price           0
Discount Percentage      0
Discount Amount          0
Categories               0
Country                  0
dtype: int64


In [168]:
data['Condition'].unique()

array(['Excellent - Refurbished', 'Used', 'Very Good - Refurbished',
       'Certified - Refurbished', 'Open box', 'New', 'Good - Refurbished',
       'Opened – never used', 'Brand New', 'For parts or not working',
       'Eccellente - Ricondizionato', 'Molto buono - Ricondizionato',
       'Buono - Ricondizionato', 'Nuovo', 'Gebraucht',
       'Hervorragend - Refurbished', 'Occasion', 'Non gradée', 'Neuf',
       'Bon état', 'Pour pièces détachées/ne fonctionne pas',
       'Ouvert (jamais utilisé)', nan, 'Comme neuf',
       'Neu: Sonstige (siehe Artikelbeschreibung)', 'Neu',
       'Sehr gut - Refurbished', 'Usato', 'Non gradata',
       'Nuovo: altro (vedi dettagli)', 'Opgeknapt door de verkoper',
       'Gut - Refurbished', 'Parfait état - Reconditionné',
       'New – Open box', 'Ungraded', 'Remanufactured', 'New with tags',
       'Per parti di ricambio o non funzionante', 'Very Good', 'Like New',
       'Très bon état - Reconditionné'], dtype=object)

In [169]:
condition_mapping = {
    "Excellent - Refurbished": "Refurbished",
    "Very Good - Refurbished": "Refurbished",
    "Certified - Refurbished": "Refurbished",
    "Good - Refurbished": "Refurbished",
    "Hervorragend - Refurbished": "Refurbished",
    "Sehr gut - Refurbished": "Refurbished",
    "Parfait état - Reconditionné": "Refurbished",
    "Très bon état - Reconditionné": "Refurbished",

    "New": "New",
    "Brand New": "New",
    "Neuf": "New",
    "Nuovo": "New",
    "Neu": "New",
    
    "Open box": "Open Box",
    "New - Open box": "Open Box",
    "Opened - never used": "Open Box",
    "Ouvert (jamais utilisé)": "Open Box",
    "Nuovo: altro (vedi dettagli)": "Open Box",
    "Neu: Sonstige (siehe Artikelbeschreibung)": "Open Box",

    "Used": "Used",
    "Gebraucht": "Used",
    "Usato": "Used",
    "Occasion": "Used",
    "Bon état": "Used",
    "Comme neuf": "Used",

    "Remanufactured": "Refurbished",
    "Opgeknapt door de verkoper": "Refurbished",
    "Non gradée": "Ungraded",
    "Non gradata": "Ungraded",
    "Ungraded": "Ungraded",

    "Per parti di ricambio o non funzionante": "For Parts",
    "Pour pièces détachées/ne fonctionne pas": "For Parts",
    "For parts or not working": "For Parts",

    "Acceptable": "Used",
    "Very Good": "Used",
    "Like New": "Used",
    "Eccellente - Ricondizionato": "Refurbished",
    "Molto buono - Ricondizionato": "Refurbished",
    "Buono - Ricondizionato": "Refurbished",
    "Gut - Refurbished": "Refurbished",
}

# Apply mapping to the DataFrame
data["Condition"] = data["Condition"].map(condition_mapping).fillna("Unknown")

# Display unique normalized conditions
print(data["Condition"].unique())


['Refurbished' 'Used' 'Open Box' 'New' 'Unknown' 'For Parts' 'Ungraded']


In [170]:
from sklearn.preprocessing import LabelEncoder
# Encode Condition Column
le = LabelEncoder()
data['Condition_encoded'] = le.fit_transform(data['Condition'].astype(str))

# Handle missing values by setting them as NaN
data.loc[data['Condition'].isna(), 'Condition_encoded'] = np.nan

# Apply KNN Imputer
imputer = KNNImputer(n_neighbors=5)
data[['Condition_encoded']] = imputer.fit_transform(data[['Condition_encoded']])

# Convert back to original category labels
data['Condition'] = le.inverse_transform(data['Condition_encoded'].astype(int))

# Drop the encoded column if no longer needed
data.drop(columns=['Condition_encoded'], inplace=True)


In [171]:
data.isnull().sum()

Title                    0
Price                    0
Currency                 0
Condition                0
Seller Feedback %        0
Seller Feedback Score    0
Shipping Cost            0
Shipping Currency        0
Original Price           0
Discount Percentage      0
Discount Amount          0
Categories               0
Country                  0
dtype: int64

In [172]:
exchange_rates = {
    'USD': 10.02,  # US Dollar
    'CAD': 7.76,   # Canadian Dollar
    'GBP': 13.85,  # British Pound
    'AUD': 6.68,   # Australian Dollar
    'EUR': 11.63,  # Euro
    'CHF': 12.40,  # Swiss Franc
    'HKD': 1.28,   # Hong Kong Dollar
    'PLN': 2.50,   # Polish Zloty
    'SGD': 7.35    # Singapore Dollar
}

# Function to convert amounts to MAD
def convert_to_mad(row):
    rate = exchange_rates.get(row['Currency'])
    row['Price'] = row['Price'] * rate
    row['Shipping_Cost'] = row['Shipping Cost'] * rate
    row['Original Price'] = row['Original Price'] * rate
    row['Discount Amount'] = row['Discount Amount'] * rate
    return row

# Apply the conversion function to each row
data = data.apply(convert_to_mad, axis=1)
data[['Currency', 'Shipping Currency']] = 'MAD'

In [173]:
import re
brands = ['hp', 'dell', 'lenovo', 'asus', 'acer', 'msi', 'apple', 'samsung', 'lg', 'chuwi', 'clevo']

# Function to extract brand
def extract_brand(title):
    title = title.lower()  # Convert title to lowercase
    for brand in brands:
        if brand in title:
            return brand.capitalize()  # Capitalize first letter (e.g., 'hp' → 'HP')
    return "Unknown"

# Function to extract processor (Intel i3/i5/i7/i9, Ryzen)
def extract_processor(title):
    title = title.lower()  # Convert to lowercase
    match = re.search(r'(i[3579]-\d+|i[3579]\b|ryzen \d+)', title, re.IGNORECASE)
    return match.group() if match else "Unknown"

# Function to extract RAM size
def extract_ram(title):
    title = title.lower()  # Convert to lowercase
    
    # Look for patterns like "32GB RAM" or just "32GB"
    match = re.search(r'(\d+)gb(?:\s*ram)?', title, re.IGNORECASE)  
    
    if match:
        ram_size = int(match.group(1))  # Convert to integer
        if ram_size < 64:  # Only consider RAM less than 64GB
            return f"{ram_size}GB"
    
    return "Unknown"

# Apply functions
data['Brand'] = data['Title'].apply(extract_brand)
data['Processor'] = data['Title'].apply(extract_processor)
data['RAM'] = data['Title'].apply(extract_ram)

# Display result
data.head()

,Title,Price,Currency,Condition,Seller Feedback %,Seller Feedback Score,Shipping Cost,Shipping Currency,Original Price,Discount Percentage,Discount Amount,Categories,Country,Shipping_Cost,Brand,Processor,RAM
0,"HP Desktop Computer PC i7, up to 64GB RAM, 4TB...",3487.5612,MAD,Refurbished,100.0,14745,0.0,MAD,17437.7058,80,13950.1446,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States,0.0,Hp,i7,Unknown
1,"Dell Desktop Computer PC i5, up to 32GB RAM, 3...",2895.7800,MAD,Refurbished,100.0,14745,0.0,MAD,14478.9000,80,11583.1200,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States,0.0,Dell,i5,32GB
2,"DELL Desktop Computer PC i7, up to 64GB RAM, 1...",1620.2340,MAD,Refurbished,99.2,2206,0.0,MAD,7715.4000,79,6095.1660,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States,0.0,Dell,i7,Unknown
3,"DELL Desktop Computer PC i7, up to 64GB RAM 16...",3566.6190,MAD,Refurbished,99.2,2206,0.0,MAD,16983.9000,79,13417.2810,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States,0.0,Dell,i7,Unknown
4,HP EliteDesk 800 G3 35W Mini PC Computer - Int...,586.0698,MAD,Used,99.9,36320,0.0,MAD,586.0698,0,0.0000,"PC Desktops & All-In-Ones, Computers/Tablets &...",eBay United States,0.0,Hp,i5-6500,8GB


In [174]:
data['Processor'].unique()

array(['i7', 'i5', 'i5-6500', 'Unknown', 'ryzen 5', 'i3-8100', 'i3',
       'i5-7500', 'i5-8500', 'i5-7300', 'i3-12100', 'i7-6700', 'i5-10400',
       'i7-8', 'i7-8700', 'i9-9900', 'i5-9500', 'i5-6400', 'ryzen 3',
       'i7-7700', 'i5-12450', 'i7-5500', 'i7-14700', 'i7-4770', 'i5-3470',
       'i5-12400', 'i3-6100', 'i5-7600', 'i5-6600', 'i3-1115', 'i5-8265',
       'i5-4690', 'i5-8400', 'ryzen 7', 'i5-6260', 'i5-430', 'i7-2600',
       'i3-8', 'i3-1315', 'i5-4590', 'i7-3770', 'i7-6600', 'i7-12700',
       'i9-12900', 'i7-3370', 'i5-10500', 'i5-7200', 'i5-6300',
       'i7-10700', 'i5-1135', 'i3-3220', 'i5-10210', 'i5-12500'],
      dtype=object)

In [175]:
def normalize_cpu(cpu):
    cpu = cpu.lower()  # Convert to lowercase for uniformity
    match = re.match(r'(i[3-9]|ryzen\s?[3-9])', cpu)  # Extracts 'i3', 'i5', 'i7', etc.
    return match.group(0) if match else 'Unknown'

data['Processor'] = data['Processor'].apply(normalize_cpu)


In [176]:
data['Processor'].unique()

array(['i7', 'i5', 'Unknown', 'ryzen 5', 'i3', 'i9', 'ryzen 3', 'ryzen 7'],
      dtype=object)

In [177]:
# categorizing categories column as 'Desktop' or 'Laptop'
def extract_category(category):
    category = category.lower()  # Convert to lowercase for accurate matching
    if 'desktops' in category:
        return 'Desktop'
    elif 'laptops' in category:
        return 'Laptop'
    else:
        return 'Other or unknown'  # If neither "desktops" nor "laptops" is found

# Apply function
data['Device Type'] = data['Categories'].apply(extract_category)
data.drop(columns=['Categories'], inplace=True)


In [178]:
data.head()

,Title,Price,Currency,Condition,Seller Feedback %,Seller Feedback Score,Shipping Cost,Shipping Currency,Original Price,Discount Percentage,Discount Amount,Country,Shipping_Cost,Brand,Processor,RAM,Device Type
0,"HP Desktop Computer PC i7, up to 64GB RAM, 4TB...",3487.5612,MAD,Refurbished,100.0,14745,0.0,MAD,17437.7058,80,13950.1446,eBay United States,0.0,Hp,i7,Unknown,Desktop
1,"Dell Desktop Computer PC i5, up to 32GB RAM, 3...",2895.7800,MAD,Refurbished,100.0,14745,0.0,MAD,14478.9000,80,11583.1200,eBay United States,0.0,Dell,i5,32GB,Desktop
2,"DELL Desktop Computer PC i7, up to 64GB RAM, 1...",1620.2340,MAD,Refurbished,99.2,2206,0.0,MAD,7715.4000,79,6095.1660,eBay United States,0.0,Dell,i7,Unknown,Desktop
3,"DELL Desktop Computer PC i7, up to 64GB RAM 16...",3566.6190,MAD,Refurbished,99.2,2206,0.0,MAD,16983.9000,79,13417.2810,eBay United States,0.0,Dell,i7,Unknown,Desktop
4,HP EliteDesk 800 G3 35W Mini PC Computer - Int...,586.0698,MAD,Used,99.9,36320,0.0,MAD,586.0698,0,0.0000,eBay United States,0.0,Hp,i5,8GB,Desktop


In [179]:
data['Country'].unique()

array(['eBay United States', 'eBay Canada (English)', 'eBay UK',
       'eBay Australia', 'eBay Austria', 'eBay Belgium (French)',
       'eBay France', 'eBay Germany', 'eBay Italy',
       'eBay Belgium (Dutch)', 'eBay Netherlands', 'eBay Spain',
       'eBay Switzerland', 'eBay Hong Kong', 'eBay Ireland',
       'eBay Canada (French)', 'eBay Poland', 'eBay Singapore'],
      dtype=object)

In [149]:
# remove 'eBay ' prefix
data['Country'] = data['Country'].str.replace('eBay ', '', regex=True)

# remove any occurrences of ' (English)' or similar patterns
data['Country'] = data['Country'].str.replace(r' \(.+\)', '', regex=True)

In [150]:
data.head()

,Title,Price,Currency,Condition,Seller Feedback %,Seller Feedback Score,Shipping Cost,Shipping Currency,Original Price,Discount Percentage,Discount Amount,Country,Shipping_Cost,Brand,Processor,RAM,Device Type
0,"HP Desktop Computer PC i7, up to 64GB RAM, 4TB...",3487.5612,MAD,Refurbished,100.0,14745,0.0,MAD,17437.7058,80,13950.1446,United States,0.0,Hp,i7,Unknown,Desktop
1,"Dell Desktop Computer PC i5, up to 32GB RAM, 3...",2895.7800,MAD,Refurbished,100.0,14745,0.0,MAD,14478.9000,80,11583.1200,United States,0.0,Dell,i5,32GB,Desktop
2,"DELL Desktop Computer PC i7, up to 64GB RAM, 1...",1620.2340,MAD,Refurbished,99.2,2206,0.0,MAD,7715.4000,79,6095.1660,United States,0.0,Dell,i7,Unknown,Desktop
3,"DELL Desktop Computer PC i7, up to 64GB RAM 16...",3566.6190,MAD,Refurbished,99.2,2206,0.0,MAD,16983.9000,79,13417.2810,United States,0.0,Dell,i7,Unknown,Desktop
4,HP EliteDesk 800 G3 35W Mini PC Computer - Int...,586.0698,MAD,Used,99.9,36320,0.0,MAD,586.0698,0,0.0000,United States,0.0,Hp,i5,8GB,Desktop


In [151]:
# save processed data
data.to_csv("output.csv", index=False)